<br/>Miniature Detection<br/>Predicting and checking YOLO results
===

These scripts can be used to detect, check and correct, if necessary, data from YOLOv8 detections. 

**Warning**

The following scripts have been created to process data from manifest IIIF following the download protocol set up in the '0_Download_processing.ipynb' notebook.
These scripts are not designed for local processing of data or data for which no URL is available.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oriflamms/LivreQuanti2025/blob/main/DataModelling/Predicting_with_YOLO.ipynb)

# Environment

## Install

In [13]:
!pip install ultralytics

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 3.4 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/216.1 MB 5.6 MB/s eta 0:00:39
   ---------------------------------------- 1.0/216.1 MB 3.0 MB/s eta 0:01:13
   ---------------------------------------- 1.6/216.1 MB 3.2 MB/s eta 0:01:07
   ---------------------------------------- 2.4/216.1 MB 3.2 MB/s eta 0:01:07
   ---------------------------------------- 2.6/216.1 MB 3.0 MB/s eta 0:01:13
    -----------------

  You can safely remove it manually.


## Import

In [30]:
%%time

import os, cv2, json, subprocess, tarfile, time, unicodedata
import uuid
from PIL import Image
from ultralytics import YOLO
from datetime import datetime
import pandas as pd

CPU times: total: 0 ns
Wall time: 0 ns


## Variables

In [33]:
print(os.getcwd())

C:\Users\stutzmann\Github\LivreQuanti2023\DataModelling


# Functions

## Helper functions

Functions to handle the labels of annotations and cope if the otherwise required "labels.txt" file is missing

In [38]:
def normalize_filename(filename):
    """
    Normalize the filename to remove special characters and ensure consistency.
    """
    return unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode('ASCII')

def get_labels(labels_file):
    '''
    This function checks if the file 'labels.txt' exists. 
    If not, it generates a .txt file with generic names for each existing class "class1" to "classN". 
    The users can then change the names later.
    Beware: if defined classes have not been used in the training dataset, they will not appear in this labels.txt file.
    '''
    
    labels_dict = {}

    if not os.path.exists(labels_file):
        print(f"{labels_file} does not exist. Generating generic class names.")
        # Assume the number of classes is known, here it's set to 10 as an example.
        num_classes = 10
        with open(labels_file, 'w') as f:
            for i in range(1, num_classes + 1):
                f.write(f"'class{i}': 'class{i}'\n")
        
        # Populate labels_dict with generic class names
        for i in range(1, num_classes + 1):
            labels_dict[f'class{i}'] = f'class{i}'
    
    else:
        # Read the existing labels file and populate the dictionary
        with open(labels_file, 'r') as labels:
            for line in labels:
                key, value = line.strip().split(': ')
                key = key.strip("'")
                value = value.strip("',\n")
                labels_dict[key] = value
    
    return labels_dict


def get_class_name(class_id, labels):
    
    """
    This function returns the class name from the class ID. If the class key is not specified, the function returns "class unknown".
    The function will be used in the 'yolo_to_csv' function.
    
    The 'class_id' parameter is the ID of the class that will return the name of the class. will be automatically filled in 'yolo_to_csv function'.
    """
    labels = labels
    return labels.get(str(class_id), 'unknown-class')

def get_class_code(class_name, labels):
    
    """
    This function returns  the the ID (key number) from the class name. If the ID key is not specified,
    the function returns "class unknown".
    
    The 'class_id' parameter is  of the class name that will return the ID of the class.
    The parameter will be automatically filled in 'generate_corrected_files' with the results 
    data from Label Studio's corrected csv file.
    """

    labels = {str([value]): key for key, value in labels.items()}
    return labels.get(str(class_name), 'unknown-class')

Functions to handle the coordinates

In [41]:

def from_relative_coordinates_to_absolute(x_center, y_center, width, height, img_width, img_height):
    """
    The function will be used in the 'yolo_to_csv' function to transform the relative coordinates of the 
    YOLO bounding box detection into absolute coordinates.
    The absolute coordinates will be used to create the URL of the bounding boxes of the detected objects.
    
    The 'x_center' parameter is the relative x coordinate of the center of the bounding box.
    The 'y_center' parameter is the relative y coordinate of the center of the bounding box.
    The 'width' parameter is the relative width of the bounding box.
    The 'height' parameter is the relative height of the bounding box.
    The 'img_width' parameter is the width of the downloaded image.
    The'img_height' parameter is the height of the downloaded image.
    
    All the parameters will be automatically filled in 'yolo_to_csv function' by the results data from the YOLO .txt files.
    """
    
    abs_x_center = x_center * img_width
    abs_y_center = y_center * img_height
    abs_width = width * img_width
    abs_height = height * img_height

    upper_left_x = abs_x_center - (abs_width / 2)
    upper_left_y = abs_y_center - (abs_height / 2)

    absolute_coordinates = int(upper_left_x), int(upper_left_y), int(abs_width), int(abs_height)
    
    return absolute_coordinates


# Data and model parameters

## Image data: choose your group!

In [45]:
while True:
    try:
        group = int(input("Please enter an integer from 1 to 7: "))
        if 1 <= group <= 7:
            break
        else:
            print("The number must be between 1 and 7. Please try again.")
    except ValueError:
        print("That's not an integer. Please try again.")

if group == 1: 
    mss = ["Besancon_AD_Doubs_MS4"]   # Group 1
    img_tar_url = 'https://box.hu-berlin.de/f/03d48ce6cb504402980e/?dl=1'
elif group == 2:
    mss = ["Besancon_BM_50"]  # Group 2
    img_tar_url = 'https://box.hu-berlin.de/f/a624527252e14e40ad83/?dl=1'
elif group == 3:
    mss = ["Besancon_BM_125"]    # Group 3
    img_tar_url = 'https://box.hu-berlin.de/f/907ff0e1497843679dcf/?dl=1'
elif group == 4:
    mss = ["Besancon_BM_136"]      # Group 4
    img_tar_url = 'https://box.hu-berlin.de/f/58e1cb7ae9e24a199ea2/?dl=1'
elif group == 5:
    mss = ["Besancon_BM_144"]  # Group 5
    img_tar_url = 'https://box.hu-berlin.de/f/8b5713a467924226b2e4/?dl=1'
elif group == 6:
    mss = ["Besancon_BM_147"]   # Group 6
    img_tar_url = 'https://box.hu-berlin.de/f/a641cdd84f604dcfb15e/?dl=1'
elif group == 7:
    mss = ["Besancon_BM_151"]  # Group 7
    img_tar_url = 'https://box.hu-berlin.de/f/1d89b0dd26f64167ad61/?dl=1'

img_data_url = 'https://box.hu-berlin.de/f/ee5257238e4f4d7d8226/?dl=1'    
    
print(f"You entered: {group}. You will work with the following manuscripts: {mss}")


Please enter an integer from 1 to 7:  1


You entered: 1. You will work with the following manuscripts: ['Besancon_AD_Doubs_MS4']


## Retrieve the image data

In [51]:
%%time

img_dir = "./mss_img/"

# Create the /data/ directory if it doesn't exist
os.makedirs(img_dir, exist_ok=True)

# Step 1: Download the tar.gz file using wget
tar_file_path = os.path.join(img_dir, f"mss_img_group{group}.tar.gz")
img_data_file_path = os.path.join(img_dir, f"mss_img_image_data.csv")


# Download the file using wget
subprocess.run(["wget", "-O", tar_file_path, img_tar_url], check=True)
subprocess.run(["wget", "-O", img_data_file_path, img_data_url], check=True)

# Step 2: Extract the tar.gz file
with tarfile.open(tar_file_path, "r:gz") as tar:
    tar.extractall(path=img_dir)


print("Download and extraction completed successfully.")


FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable

## Retrieve the miniature detection model

In [10]:
%%time
model_url = 'https://box.hu-berlin.de/f/277985af79314c79963e/?dl=1'
model_dir = "./model/"
model_name = "HORAE_Images_Folio_Miniatures_20240526_x_i640_e120_b8_w24.pt"

os.makedirs(model_dir, exist_ok=True)
model_weights_file_path = os.path.join(model_dir, model_name)


CPU times: user 686 μs, sys: 0 ns, total: 686 μs
Wall time: 1.17 ms


In [11]:

# Download the file using wget
subprocess.run(["wget", "-O", model_weights_file_path, model_url], check=True)

--2025-04-09 11:36:11--  https://box.hu-berlin.de/f/277985af79314c79963e/?dl=1
Resolving box.hu-berlin.de (box.hu-berlin.de)... 141.20.184.42
Connecting to box.hu-berlin.de (box.hu-berlin.de)|141.20.184.42|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://box.hu-berlin.de/seafhttp/files/4ac06928-226d-401b-9886-bb966c1a9098/HORAE_Images_Folio_Miniatures_20240526_x_i640_e120_b8_w24.pt [following]
--2025-04-09 11:36:12--  https://box.hu-berlin.de/seafhttp/files/4ac06928-226d-401b-9886-bb966c1a9098/HORAE_Images_Folio_Miniatures_20240526_x_i640_e120_b8_w24.pt
Reusing existing connection to box.hu-berlin.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 546045599 (521M) [application/octet-stream]
Saving to: ‘./model/HORAE_Images_Folio_Miniatures_20240526_x_i640_e120_b8_w24.pt’

     0K .......... .......... .......... .......... ..........  0%  245K 36m19s
    50K .......... .......... .......... .......... ..........  0% 1.38M 21m18s
   100K

CompletedProcess(args=['wget', '-O', './model/HORAE_Images_Folio_Miniatures_20240526_x_i640_e120_b8_w24.pt', 'https://box.hu-berlin.de/f/277985af79314c79963e/?dl=1'], returncode=0)

# Predicting with YOLO

## Define corpus to be processed

* Variables to be changed

```
dataset_path = 'ABSPATHTOTHEFOLDER'  # (to be changed, asbolute path to a folder with images only, without annotations.)
yolo_model_folder = 'ABSPATHTOTHEMODELFOLDER' 
corrected_predictions_folder_to_be_excluded = ''

```
In other implementations, to be changed and adapted, using asbolute paths to the folder with the model, typically ```/home/jovyan/work/runs/train/{model_name}```. The following code will use the weights at the path ```{model_name}/weights/best.pt```

Here, relative paths corresponding to the data and model that have just been retrieved.


In [12]:
dataset_path ='./mss_img' # to be changed, absolute or relative path to a folder with images only, without annotations.
yolo_model_folder = f'./model'
yolo_model_name = model_name
# yolo_model_folder = '/home/jovyan/work/runs/train/Miniatures_new_classes_20230916_l_i640_e100_b8_w24' # to be changed, asbolute path to the folder with the training data



## Prediction script

Source : 


Documentation : https://github.com/ultralytics/ultralytics/issues/2143


In [13]:
def process_images_with_yolo(yolo_model_folder, dataset_path):
    """
    Function to process all image files in a folder and its subfolders recursively
    """
    
    # print(type(corr_files))
    
    for root, dirs, files in os.walk(dataset_path):
        
        # Exclude hidden folders (i.e folders whose names start with ".")
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        i=0
        l=len(files)
        
        files.sort()
        
        for filename in files: 
            if filename.lower().endswith(('.jpg', '.png')):
                image_path = os.path.join(root, filename)
                process_single_image_with_yolo(yolo_model_folder, dataset_path, image_path)
                print(f'''{i}/{l}, {dirs} {image_path}''')
            else:
                print('Excluded : ', filename, 'has been excluded')
            i+=1

def process_single_image_with_yolo(yolo_model_folder, dataset_path, image_path): #suppr time_sleep
    """
    This function makes predictions using YOLO for the various files returned thanks to predict_on_dataset.
    """
    yolo_model_path = os.path.join(yolo_model_folder, model_name)
    yolo_model = YOLO(yolo_model_path)
    
    output_directory = os.path.join(
        os.path.dirname(yolo_model_folder), 
        'predict', 
        os.path.dirname(os.path.relpath(image_path, dataset_path)),
        'labels'        
    )
    print(output_directory)
    os.makedirs(output_directory, exist_ok=True)
    
    image = cv2.imread(image_path)

    # Process the image using YOLO
    results = yolo_model.predict(source=image,
                       imgsz=640,
                       # name=os.path.basename(output_directory), # os.path.basename(output_directory), # Use the project folder as the name of the project
                       save_txt=False,
                       save_conf=False,
                       # project=output_directory
                      )
    
    boxes = results[0].boxes
    
    if not boxes:
            print(f"No detections found in {os.path.basename(image_path)}")
            return
    
    # Save the labels in the /labels/ folder
    label_filename = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
    label_path = os.path.join(output_directory, label_filename)
    
    with open(label_path, 'w') as label_file:
        for box in boxes:
            xywh = " ".join([f"{value:.4f}" for value in box.xywhn.cpu().squeeze().tolist()])
            label_data = f'''{box.cls.cpu().item()} {xywh} {box.conf.cpu().item()}\n'''
            label_file.write(label_data)
        
        


## <u>/!\ Launch Yolo

In [14]:
%%time

process_images_with_yolo(yolo_model_folder, dataset_path)

Excluded :  mss_img_group1.tar.gz has been excluded
Excluded :  mss_img_image_data.csv has been excluded
./predict/IV D 5/labels

0: 640x480 (no detections), 388.0ms
Speed: 7.6ms preprocess, 388.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 480)
No detections found in IV D 5_1.jpg
0/329, [] ./mss_img/IV D 5/IV D 5_1.jpg
./predict/IV D 5/labels

0: 640x96 (no detections), 100.5ms
Speed: 0.4ms preprocess, 100.5ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 96)
No detections found in IV D 5_10.jpg
1/329, [] ./mss_img/IV D 5/IV D 5_10.jpg
./predict/IV D 5/labels

0: 640x448 (no detections), 283.3ms
Speed: 3.8ms preprocess, 283.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 448)
No detections found in IV D 5_100.jpg
2/329, [] ./mss_img/IV D 5/IV D 5_100.jpg
./predict/IV D 5/labels

0: 640x448 (no detections), 273.4ms
Speed: 2.7ms preprocess, 273.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 448)
No detections found in IV D

# Visualize results: generate IIIF files for IIIF corpora (csv, html)

## Generate overview of results (CSV files)

In [15]:
def yolo_to_csv(dataset_path, yolo_model_folder):
    """
    This function generates one or several CSV files: 
    - one for the entire corpus if all images in the same folder,
    - one for each manuscript, if there are several folders within the corpus, and one for the entire corpus
    It also retrieves the information on the images based on the CSV _image_data.csv file that was generated when downloading the images from IIIF manifests
    """
    for root, dirs, files in os.walk(dataset_path):
        dirs[:] = [d for d in dirs if not d.startswith('.')]  # Ignore folders starting with '.'
        
        if dirs == []:
            labels_folder = os.path.join(
                os.path.dirname(os.path.dirname(yolo_model_folder)),
                'predict',
                os.path.basename(dataset_path),
                'labels')

        else:
            for dir in dirs:

                # (1) Retrieve informations on images
                csv_file = [file for file in os.listdir(os.path.join(root, dir)) if file.lower().endswith('_image_data.csv')]
                print(csv_file)
                images_data = pd.read_csv(os.path.join(root, dir, csv_file[0]), sep=',')
                

                # (2) Retrieve YOLO annotations at manuscript level: Search for annotation files (.txt files) in the labels folder
                
                labels_folder = os.path.join(
                    os.path.dirname(yolo_model_folder), 
                    'predict', 
                    os.path.basename(dir),
                    'labels')

                annotation_files = [file for file in os.listdir(labels_folder) if file.endswith('.txt')]
                print(f"""Labels in : {labels_folder}. There are {len(annotation_files)} annotations""")

                # Check for annotations
                if len(annotation_files) == 0:
                    print(f'No detection on the data set {labels_folder}.')

                else:
                    results_folder = os.path.join(labels_folder.replace('labels', 'results'))
                    os.makedirs(results_folder, exist_ok=True)


                # (3) Compare infos on image files and YOLO annotations files with images and create a CSV overview file. 
                # The following code takes the image list and look for annotations rather than taking a YOLO annotation file and looking up in the pandas dataframe 

                rows = []

                for _, row in images_data.iterrows():
                    image_path = row["imageFileName"]
                    image_width = row["imageWidthAsDownloaded"]
                    image_height = row["imageHeightAsDownloaded"]
                    image_url = row['urlImage']

                    # Check whether the image corresponds to an annotation (with standardised names to ensure consistency)
                    matching_annotations = [annotation_file for annotation_file in annotation_files if normalize_filename(os.path.basename(image_path)) == normalize_filename(os.path.basename(annotation_file)).replace('txt', 'jpg')]

                    for matching_annotation in matching_annotations:
                        with open(os.path.join(labels_folder, matching_annotation), 'r') as f:
                            for line in f.readlines():
                                class_id, x_center, y_center, width, height, confidence = map(float, line.split())
                                x, y, abs_width, abs_height = from_relative_coordinates_to_absolute(x_center, y_center, width, height, image_width, image_height)

                                # Create a line of data for the DataFrame
                                rows.append({
                                    'Image_Path': image_path,
                                    'Image_Width': image_width,
                                    'Image_Height': image_height,
                                    'YOLO_Results_File': os.path.join(labels_folder, matching_annotation),
                                    'Class_Id': int(class_id),
                                    'Class_Name': get_class_name(int(class_id), get_labels(os.path.join(yolo_model_folder, 'labels.txt'))),
                                    'Detected_coordinates': f'{x_center} {y_center} {width} {height}',
                                    'Absolute_coordinates': f"{x} {y} {abs_width} {abs_height}",
                                    'Confidence': confidence,
                                    'Url_Detection': image_url.replace("full", f"{x},{y},{abs_width},{abs_height}", 1),
                                    'Url_Image': image_url
                                })

                # Create a Pandas DataFrame from the data and save the output CSV file
                if len(rows) == 0:
                    print(f"No correspondence found.")
                else:
                    df = pd.DataFrame(rows)
                    df_sorted = df.sort_values('Image_Path')

                    if dirs == []:
                        df_sorted.to_csv(os.path.join(results_folder, os.path.basename(dataset_path) + '.csv'), index=False)
                        print(f"The file {os.path.join(results_folder, os.path.basename(dataset_path) + '.csv')} has been created")
                    else:
                        df_sorted.to_csv(os.path.join(results_folder, os.path.basename(dir) + '.csv'), index=False)
                        print(f"The file {os.path.join(results_folder, os.path.basename(dir) + '.csv')} has been created")
            
            
            # (5) Create an overview CSV file with all predicted results
            results_folder = os.path.join(
                    os.path.dirname(yolo_model_folder), 
                    'predict'
            )
            
            csv_files = []

            # Parcourir les dossiers et sous-dossiers
            for root, dirs, files in os.walk(results_folder):
                # Vérifier si le dossier courant est un dossier "results"
                if os.path.basename(root) == "results":
                    # Récupérer tous les fichiers CSV dans le dossier "results"
                    csv_files.extend([os.path.join(root, file) for file in files if file.endswith('.csv')])

            if not csv_files:
                print("Aucun fichier CSV trouvé dans les dossiers 'results'.")
                return

            # Concaténer les fichiers CSV
            dfs = []
            for csv_file in sorted(csv_files):  # Triez les fichiers CSV par ordre alphabétique
                df = pd.read_csv(csv_file)
                dfs.append(df)

            concatenated_df = pd.concat(dfs, ignore_index=True)

            # Écrire le DataFrame concaténé dans un nouveau fichier CSV
            concatenated_csv_path = os.path.join(results_folder, 'results', f"{os.path.basename(dataset_path)}.csv")
            os.makedirs(os.path.join(results_folder, 'results'), exist_ok=True)
            concatenated_df.to_csv(concatenated_csv_path, sep=';', index=False)

            print(f"CSV files in 'results' folders were successfully concatenated to '{concatenated_csv_path}'.")

        

In [16]:
%%time
yolo_to_csv(dataset_path, yolo_model_folder)
# yolo_to_csv_booksinminiatures(dataset_path, yolo_model_folder) #for results only for images with predicted annotations + with coordinates on the second grade (annotation > miniature > image)

['IV D 5_image_data.csv']
Labels in : ./predict/IV D 5/labels. There are 36 annotations
./model/labels.txt does not exist. Generating generic class names.
The file ./predict/IV D 5/results/IV D 5.csv has been created
['XII F 42 _image_data.csv']
Labels in : ./predict/XII F 42 /labels. There are 3 annotations
The file ./predict/XII F 42 /results/XII F 42 .csv has been created
['I E 38_image_data.csv']
Labels in : ./predict/I E 38/labels. There are 6 annotations
The file ./predict/I E 38/results/I E 38.csv has been created
CSV files in 'results' folders were successfully concatenated to './predict/results/mss_img.csv'.
CPU times: user 51.4 ms, sys: 8.87 ms, total: 60.3 ms
Wall time: 183 ms


## Generate overview of results (html file)

In [17]:
def generate_html_with_and_without_mouseover(dataset_path, model_folder, sort_by=None):
    base_path = os.path.dirname(os.path.commonprefix([dataset_path, model_folder]))
    dataset_name = os.path.basename(dataset_path)
    results_path = os.path.join(base_path, 'predict')

    print(results_path)
    csv_result = os.path.join(results_path, 'results', dataset_name + '.csv')

    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
      <title>""" + f"{dataset_name} (Yolo v8 Predictions with model {model_name}" + """</title>
      <style>
        body {
          display: flex;
          flex-wrap: wrap;
        }

        img {
          max-height: 320px;
          flex: 0 0 auto;
          margin: 10px;
        }
      </style>
    </head>
    <body>
      <h1>""" + f"{dataset_name} <br/> (Yolo v8 Predictions with model {model_name}" + """</h1>

      <div id="image-container"></div>

      <script>
        var imageUrls = [{{image_urls}}];
        var imageInfos = {{image_infos}};
        var fullImageUrls = [{{full_image_urls}}];

        var imageContainer = document.getElementById("image-container");
        for (var i = 0; i < imageUrls.length; i++) {
          var imageUrl = imageUrls[i];
          var info = imageInfos[i];
          var fullImageUrl = fullImageUrls[i];

          var linkElement = document.createElement("a");
          linkElement.href = fullImageUrl;
          linkElement.target = "_blank"; // Open in a new tab

          var imgElement = document.createElement("img");
          imgElement.src = imageUrl;
          imgElement.title = info; // Show the info as tooltip on mouseover

          linkElement.appendChild(imgElement);
          imageContainer.appendChild(linkElement);
        }
      </script>
    </body>
    </html>
    """

    if not os.path.exists(csv_result):
        print('No detection on the data set, the HTML file has not been created.')
        print('You do not need to continue.')
    else:
        df = pd.read_csv(csv_result, sep=';')
        
        # Optional sorting based on the specified column
        if sort_by is not None and sort_by in df.columns:
            df = df.sort_values(by=sort_by)

        # Extract URLs and infos
        image_urls = df["Url_Detection"].tolist()
        image_infos = df["Image_Path"].tolist()
        full_image_urls = df["Url_Image"].tolist()

        # Convert lists to JSON arrays for JavaScript
        image_urls_str = json.dumps(image_urls)[1:-1]  # Remove outer brackets
        image_infos_str = json.dumps([os.path.basename(url) for url in image_infos])
        full_image_urls_str = json.dumps(full_image_urls)[1:-1]

        # Replace the position markers in the HTML template with the actual data
        html_content_with_mouseover = html_content.replace("{{image_urls}}", image_urls_str).replace("{{image_infos}}", image_infos_str).replace("{{full_image_urls}}", full_image_urls_str)
        html_content_without_mouseover = html_content.replace("{{image_urls}}", image_urls_str).replace("{{image_infos}}", '[""]' * len(image_urls)).replace("{{full_image_urls}}", full_image_urls_str)

        # Writing HTML content to files
        output_html_path_with_mouseover = os.path.join(results_path, 'results', dataset_name + '_' + (sort_by if sort_by else 'default') + '_with_mouseover.html')
        with open(output_html_path_with_mouseover, "w") as file:
            file.write(html_content_with_mouseover)
        print(f"The file {output_html_path_with_mouseover} has been generated with mouseover tooltips.")

        output_html_path_without_mouseover = os.path.join(results_path, 'results', dataset_name + '_' + (sort_by if sort_by else 'default') + '_without_mouseover.html')
        with open(output_html_path_without_mouseover, "w") as file:
            file.write(html_content_without_mouseover)
        print(f"The file {output_html_path_without_mouseover} has been generated without mouseover tooltips.")


In [18]:

generate_html_with_and_without_mouseover(dataset_path, yolo_model_folder, sort_by='Image_Path')

./predict
The file ./predict/results/mss_img_Image_Path_with_mouseover.html has been generated with mouseover tooltips.
The file ./predict/results/mss_img_Image_Path_without_mouseover.html has been generated without mouseover tooltips.
